# Import Packages

In [1]:
import sys
 
import os
import pandas as pd
import numpy as np
import pickle
import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt

import Lib_RWA as Lib_RWA

# delete lib_RWA from system modules 

del(sys.modules['Lib_RWA'])

# delete SFR_PD_Recalibration_2023_Lib from system modules

del(sys.modules['SFR_PD_Recalibration_2023_Lib'])


import SFR_PD_Recalibration_2023_Lib as tool

import Lib_RWA as Lib_RWA

import math

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Inputs

In [3]:

# Snaphsot and RWA file information

# snapshot_num = 202212 # 20250403 Fran commented out as George in S1
# snapshot = str(snapshot_num) # 20250403 Fran commented out as George in S1
# snapshot_date = '2022-12-31' # 20250403 Fran commented out as George in S1
# RWA_File_from_Chenxi = 'SFR_RWA_202212.csv' #20250403 Fran added as George S1!!! Note to be consistent with the snapshot date

snapshot_num = 202412 # 20250403 Fran added as George in S1 
snapshot = str(snapshot_num) # 20250403 Fran added as George in S1
snapshot_date = '2024-12-31' # 20250403 Fran added as George in S1
RWA_File_from_Chenxi = 'SFR_202412_v2 (from Chenxi 20250403).csv' #20250403 Fran added as George S1!!! Note to be consistent with the snapshot date

In [4]:
#CAR 2023 Ch5 prescribed parameters

correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023

In [ ]:
# Columns to Keep

RWA_Info_Cols_to_Keep = ['Loan_Number','Insured_class','EAD','Advance_Amount','Years_to_maturity', 'corr_uninsured', 'RWA_standardized']

PD_Cols_to_Keep = ['Loan_Number','SL_Date','PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC','MRS_Bin_PD','Insured_Ind','Alt_Prime_Indicator','RemainingPrincipal_Tot_Exp','RemainingPrincipal_Excl_Partner', 'Corporate_Applicant_Ind','Combo_Province_Metro_Override','Combo_Province_Metro_Override_WOE', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE','Beacon_Avg_App_CoApp','Beacon_Avg_App_CoApp_WOE','BF_LTV_Tot_Exp_FSA_Dw_WF','Combo_LTV_Insured_Ind','Combo_LTV_Insured_Ind_WOE']

PD_Cols_for_Corp = ['AdvancedAmount_EQB_Exp','AdvancedAmount_Total_Exp','AdvancedAmt_Incl_Part','AdvancedAmt_Excl_Part','Remaining_Term'] 


LGD_Cols_to_Keep = ['Loan_Number','MRS_Bin_LGD','LR_Avg_LGD','Segment_Avg_LGD','Base_Line_LGD','LGD_DT_Adjusted','Model_LGD','Sub_Product','BF_Appr_Prov_Dw','Appraisal_Bin_WOE', 'Occupancy', 'Occupancy_WOE', 'Province_Foreclosure', 'Province_Foreclosure_WOE', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw','LTV_Bin_WOE' ]



# Set Folder

In [6]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added


if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added




# Load RWA Data Info

In [7]:

# Read in the RWA data from Chenxi

# Import data with Loan_Number,Insured_class,CalibratedPD,Final_LGD,EAD,Advance_Amount,Years_to_maturity and corr_uninsured
##################### load snapshot data Chenxi provided and rename#################################
#rwa_data = pd.read_csv(input_dir +'\\'+'SFR_RWA_' + snapshot + '.csv', low_memory=False) #20250403 Fran commented as George in S1
rwa_data = pd.read_csv(input_dir +'\\'+ RWA_File_from_Chenxi, low_memory=False) #20250403 Fran added as George in S1

rwa_raw_data_0 = rwa_data[RWA_Info_Cols_to_Keep]  # 20250410 George add

In [8]:
rwa_data.shape

(53402, 17)

In [9]:
#Floor the EAD on 0          

rwa_raw_data_0.loc[rwa_raw_data_0['EAD']<0, ['EAD']] =0


# Load PD Data

In [10]:
# add new SFR PD model generated PD 

#new_pd_data = pd.read_csv( Active_dir +'\\'+'PARAM_Pred_PD'  + '.csv', low_memory=False) # 20250402 George Commented out 
new_pd_data = pd.read_pickle(input_dir + '\\Rating_July2020_to_Dec2024_EQB (from George 20250414).pkl') # 20250402 George added
new_pd_data_red = new_pd_data.loc[new_pd_data['SL_Date'] == snapshot_date]


In [11]:
# print columns in alphabetical order in the new_pd_data_red dataframe #20250402 George added

print(sorted(new_pd_data_red.columns)) #20250402 George added


['AIRB_PD_Risk_Rating', 'AdvancedAmount_EQB_Exp', 'AdvancedAmount_Total_Exp', 'AdvancedAmt_Excl_Part', 'AdvancedAmt_Incl_Part', 'Alt_Prime_Indicator', 'AppraisalValue_Inception', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'City', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'Delinquency_Status_Adj', 'Dwelling_Type', 'FSA', 'Funded_Date', 'Insured_Ind', 'LoanType', 'Loan_Number', 'Metro_Region_BF_FMT', 'PD_Post_MOC', 'PD_Pre_MOC', 'PD_scoring', 'Prior_24_Worse_Delinquent_Status_FMT_Adj', 'Prior_24_Worse_Delinquent_Status_FMT_Adj_WOE', 'Province', 'RemainingPrincipal_Excl_Partner', 'RemainingPrincipal_Tot_Exp', 'Remaining_Term', 'SL_Date', 'Sub_Product', 'log_odds']


In [12]:
# For columns PD_Post_MOC_Pre_Adj and PD_Pre_MOC, if not in the data set, add them with nan #20250402 George added

if 'PD_Post_MOC_Pre_Adj' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added

if 'PD_Pre_MOC' not in new_pd_data_red.columns: #20250402 George added

    new_pd_data_red['PD_Pre_MOC'] = np.nan #20250402 George added


# Rename AIRB_PD_Risk_Rating to be MRS_Bin #20250402 George added

new_pd_data_red = new_pd_data_red.rename({'AIRB_PD_Risk_Rating':'MRS_Bin'}, axis='columns')  #20250402 George added

C:\Users\georgez\AppData\Local\Temp\ipykernel_28156\1440374422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_pd_data_red['PD_Post_MOC_Pre_Adj'] = np.nan #20250402 George added


In [13]:
# Save Loan number list for Chenxi as she requested to check
with pd.ExcelWriter(Output_dir +'\\'+'eqb_loans_list_dec24.xlsx' )as writer:
    new_pd_data_red['Loan_Number'].to_excel(writer, sheet_name='loan_list', index=False)

In [14]:
# Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added
print('***************Checking Default Loans *****************') #20250402 George added
print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added



***************Checking Default Loans *****************
  MRS_Bin Loan_Number PD_Post_MOC          
              nunique         min       max
0     1.0       26705    0.002228  0.002228
1     2.0       14383    0.003622  0.003622
2     3.0        5505    0.007377  0.007377
3     4.0        3148    0.011499  0.011499
4     5.0        1751    0.019067  0.019067
5     6.0        1765    0.034554  0.034554
6     7.0        1588    0.107857  0.107857
7     8.0         402    0.629118  0.629118
8     9.0         382    1.000000  1.000000


In [15]:
# # Remove Dafault Loans

# # Remove rows with PD_Post_MOC <=0 or >=1 (defaults) #20250402 George added

# new_pd_data_red = new_pd_data_red.loc[(new_pd_data_red['PD_Post_MOC'] > 0) & (new_pd_data_red['PD_Post_MOC'] < 1)] #20250402 George added

# # Group by MRS_Bin, count distinct Loan_Number and min and max of PD_Post_MOC #20250402 George added

# print('***************After Removing Default Loans*****************') #20250402 George added

# print(new_pd_data_red.groupby(['MRS_Bin'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'PD_Post_MOC': [min, max]})) #20250402 George added

In [16]:
# Rename Columns

new_pd_data_red.rename({'MRS_Bin':'MRS_Bin_PD'}, axis = 'columns', inplace=True) #20250412 George added

## Check Corporate Loan

In [17]:
# Group by SL_Date and Corporate_Applicant_Ind, count number of Loan_Number

print('***************Group by SL_Date and Corporate_Applicant_Ind*****************') #20250403 George added

print(new_pd_data_red.groupby(['SL_Date','Corporate_Applicant_Ind'], as_index=False).agg({'Loan_Number': pd.Series.nunique})) #20250403 George added

***************Group by SL_Date and Corporate_Applicant_Ind*****************
     SL_Date  Corporate_Applicant_Ind  Loan_Number
0 2024-12-31                      0.0        54203
1 2024-12-31                      1.0         1426


In [18]:
# Compare with RWA data from Chenxi

# Outer Join new_pd_data_red with rwa_raw_data_0 on Loan_Number #20250403 George added

rwa_raw_data_pd_check = pd.merge(rwa_raw_data_0['Loan_Number'], new_pd_data_red[['Loan_Number','Corporate_Applicant_Ind']], how='outer', on=['Loan_Number'], indicator= 'merge_') #20250403 George added

# change the content to be more meaningful #20250403 George added

rwa_raw_data_pd_check['merge_'] = rwa_raw_data_pd_check['merge_'].replace({'left_only': 'Loan in Chenxi only', 'right_only': 'Loan in PD Data only', 'both': 'Loan in both'}) #20250403 George added


# group by merge_ and Corporate_Applicant_Ind, including missing values, count number of Loan_Number #20250403 George added

print('***************Group by merge_ and Corporate_Applicant_Ind*****************') #20250403 George added

print(rwa_raw_data_pd_check.groupby(['merge_','Corporate_Applicant_Ind'], dropna= False).agg({'Loan_Number': pd.Series.nunique})) #20250403 George added

***************Group by merge_ and Corporate_Applicant_Ind*****************
                                              Loan_Number
merge_               Corporate_Applicant_Ind             
Loan in Chenxi only  NaN                             30.0
                     0.0                              NaN
                     1.0                              NaN
Loan in PD Data only NaN                              NaN
                     0.0                            834.0
                     1.0                           1423.0
Loan in both         NaN                              NaN
                     0.0                          53369.0
                     1.0                              3.0


# Load LGD Data

In [19]:
# add new SFR LGD model generated LGD and apply LGD floor 

# 2023_11-04 is most recent one from Ben
#new_lgd_data = pd.read_csv(input_dir+'\Dec_2022_LGD_2023_11_04_Recalibrated_Model.csv', low_memory = False)

if snapshot_num == 202212:

    new_lgd_data = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2022_12 (from Abhi 20250410).pkl') # 20250406 George added

if snapshot_num == 202412:

    new_lgd_data = pd.read_pickle(input_dir + '\\eqb_lgd_scored_2024_12 (from Abhi 20250404).pkl') # 20250406 George added


In [20]:
# print duplicated columns 

print(new_lgd_data.columns[new_lgd_data.columns.duplicated()].tolist()) #20250412 George added

# Remove duplicated columns 

new_lgd_data = new_lgd_data.loc[:,~new_lgd_data.columns.duplicated()] #20250412 George added

['LTV_Bin_WOE', 'Appraisal_Bin_WOE']


In [21]:
# print columns in alphabetical order in the new_lgd_data dataframe 

print(sorted(new_lgd_data.columns)) #20250402 George added

['Appraisal_Bin', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'Default_Ind', 'Final_LGD', 'Foreclosure_Ind', 'Foreclosure_Ind_WOE', 'Insured_Ind', 'LR_Avg_LGD', 'LTV_Bin', 'LTV_Bin_WOE', 'LoanType', 'Loan_Number', 'MRS_Bin', 'Mapped_LGD', 'Occupancy', 'Occupancy_WOE', 'Pred_LGD', 'Province_Foreclosure', 'Province_Foreclosure_WOE', 'RemainingPrincipal_Excl_Partner', 'SL_Date', 'Sub_Product']


In [22]:
# Add columns LGD_DT_Adjusted, Segment_Avg_LGD and Base_Line_LGD if they are not in the data set #20250412 George added

if 'LGD_DT_Adjusted' not in new_lgd_data.columns: # 20250412 George added
    new_lgd_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

if 'Segment_Avg_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Segment_Avg_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

if 'Base_Line_LGD' not in new_lgd_data.columns: # 20250412 George added, same treatment as RWA for Concentra
    new_lgd_data['Base_Line_LGD'] = new_lgd_data.loc[:,'Mapped_LGD']

In [23]:
# Rename Columns

new_lgd_data = new_lgd_data.rename({'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}, axis = 'columns')


# Merge PD and LGD data for Corporate Loans

In [24]:
# join new_Pd_data_red and new_lgd_data on Loan_Number

PD_LGD_Merge = pd.merge(new_pd_data_red[PD_Cols_to_Keep + PD_Cols_for_Corp], new_lgd_data[LGD_Cols_to_Keep], how='outer', on=['Loan_Number'])

In [25]:

# Filter with Corporate_Applicant_Ind = 1 #20250404 George added

PD_LGD_Merge_Corp = PD_LGD_Merge.loc[PD_LGD_Merge['Corporate_Applicant_Ind'] == 1] #20250404 George added

# Group by Insured_Ind, MRS_Bin_PD and MRS_Bin_LGD, count number of Loan_Number and sum of RemainginPrincipal_Tot_Exp#20250404 George added

print('***************Corporate Loans Only*****************') #20250404 George added

print(PD_LGD_Merge_Corp.groupby(['Insured_Ind','MRS_Bin_PD','MRS_Bin_LGD'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'RemainingPrincipal_Tot_Exp': sum})) #20250404 George added


***************Corporate Loans Only*****************
    Insured_Ind  MRS_Bin_PD  MRS_Bin_LGD  Loan_Number  RemainingPrincipal_Tot_Exp
0   Not Insured         1.0            1          225                9.839399e+07
1   Not Insured         1.0            2           91                4.079501e+07
2   Not Insured         1.0            3           15                7.153886e+06
3   Not Insured         1.0            4            1                2.092401e+05
4   Not Insured         2.0            1          110                6.913848e+07
5   Not Insured         2.0            2          268                1.580170e+08
6   Not Insured         2.0            3          175                1.119233e+08
7   Not Insured         2.0            4           69                3.055346e+07
8   Not Insured         3.0            1           31                2.121737e+07
9   Not Insured         3.0            2          102                6.558273e+07
10  Not Insured         3.0            3     

In [26]:

# Save data to excel #20250404 George added

with pd.ExcelWriter(Output_dir +'\\'+'Corporate_Loans_Breakdown.xlsx' )as writer:

    PD_LGD_Merge_Corp.to_excel(writer, sheet_name='Corp_Loans', index=False) #20250404 George added

# Merge RWA Data Info with PD Data

In [27]:
#test = new_pd_data_red[ ['PD_Pre_MOC','PD_Post_MOC_Pre_Adj','PD_Post_MOC'] ].drop_duplicates()

rwa_raw_data_1 = pd.merge(rwa_raw_data_0, new_pd_data_red[PD_Cols_to_Keep], how='outer',on=['Loan_Number'], indicator= 'merge_Chenxi_PD')

rwa_raw_data_1['merge_Chenxi_PD'] = rwa_raw_data_1['merge_Chenxi_PD'].replace({'left_only': 'Loan in Chenxi only', 'right_only': 'Loan in PD Data only', 'both': 'Loan in both Chenxi and PD'}) #20250403 George added

# rwa_raw_data_2 =  rwa_raw_data_1.loc[~rwa_raw_data_1['PD_Pre_MOC'].isnull()] #remove records not existing in Chenxi's data since they cannot be compared

rwa_raw_data_2 = rwa_raw_data_1.copy()

rwa_raw_data_3 = rwa_raw_data_2.copy()


# Merge RWA info with LGD Data

In [28]:

#Gen3 LGD data, only including uninsured loans

rwa_raw_data_4 = pd.merge(rwa_raw_data_3,new_lgd_data[LGD_Cols_to_Keep], how='outer', on= ['Loan_Number'], indicator= 'merge_ChenxiPD_LGD')

rwa_raw_data_4['merge_ChenxiPD_LGD'] = rwa_raw_data_4['merge_ChenxiPD_LGD'].replace({'left_only': 'Loan in ChenxiPD only', 'right_only': 'Loan in LGD Data only', 'both': 'Loan in both ChenxiPD and LGD'})


# rwa_raw_data_4 = rwa_raw_data_4.rename({'MRS_Bin':'MRS_Bin_LGD','Final_LGD':'Model_LGD'}, axis = 'columns')

# Calculate RWA with 0 Addon

In [29]:
# Force Addon to be 0

rwa_raw_data_0Addon = rwa_raw_data_4.copy()
rwa_raw_data_0Addon['AddOn'] =0

In [30]:
print(CMHC_pd)

0.0001


In [31]:
# Run RWA calculation function

[df_out_0Addon, rwa_by_Insured_class_0Addon, rwa_by_MRS_Bin_0Addon, res_data_0Addon]  = Lib_RWA.rwa_calculation(df_input_data = rwa_raw_data_0Addon, lgd_gen_floor = lgd_gen_floor, CMHC_lgd = CMHC_lgd, CMHC_pd = CMHC_pd )



0
1
2


In [32]:
# print duplicated columns in res_data_0Addon - These are in LGD data set

print(res_data_0Addon.columns[res_data_0Addon.columns.duplicated()].tolist()) 


[]


In [33]:
# Save Data to parquet

res_data_0Addon.to_parquet(Output_dir +'\\'+'eqb_rwa_addon_zero.parquet', index=False)

# with pd.ExcelWriter(Output_dir +'\\'+'eqb_rwa_addon_zero.xlsx' )as writer:
#     res_data_0Addon.to_excel(writer, sheet_name='RWA_data', index=False)
#     df_out_0Addon.to_excel(writer, sheet_name="RWA") 
#     rwa_by_Insured_class_0Addon.to_excel(writer, sheet_name="rwa_by_Insured_class")
#     rwa_by_MRS_Bin_0Addon.to_excel(writer, sheet_name="rwa_by_MRS_Bin")

# Get LGD Addon and Calculate RWA

In [34]:
## retrieve DLGD floor readily available AddOn from Finance Production database
# Get max Feedid
server_2 = 'EQDWP01'  
database_2 = 'ET_Finance_Production'   



def sql_max_FeedID( snapshot ):
    
    table = 'ET_Finance_Production.dbo.tb_RE_log' # 20250410 George added

    if pd.to_datetime(snapshot) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT max([FeedID]) as max_FeedID
       FROM {table}
       where Reporting_date =  {snapshot} 
             
            '''  
    return sql

Max_Feed_ID= tool.download_from_sql( server_2, database_2, sql_max_FeedID( "'" + snapshot_date + "'" ) )['max_FeedID'][0]

def sql_LGD_addon( Max_Feed_ID ):
    
    table = 'ET_Finance_Production.dbo.tb_X_WoE_LGD_Result' # 20250410 George added

    if pd.to_datetime(snapshot_date) <= pd.to_datetime('2024-10-31'):
        table = table + '_G2'  # 20250410 George added for Generation 2 model

    sql = f'''
      SELECT 
       [LoanNumber]
       ,[Add_on_LGD] as AddOn
       FROM {table}
       where RunID =  {Max_Feed_ID} 
            '''  
    return sql

Add_on_LGD= tool.download_from_sql( server_2, database_2, sql_LGD_addon(Max_Feed_ID) )

Add_on_LGD['LoanNumber'] = Add_on_LGD['LoanNumber'].astype(str).astype(int)

rwa_raw_data_TrueAddon = pd.merge(rwa_raw_data_4, Add_on_LGD, how='left', left_on= ['Loan_Number'],right_on=['LoanNumber'])


#rwa_raw_data_5['Final_LGD'] = rwa_raw_data_5['Final_LGD_old']
#rwa_raw_data_5['PD_Post_MOC'] = rwa_raw_data_5['CalibratedPD_old']

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)
c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\SFR_PD_Recalibration_2023_Lib.py:1732: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn,**kwargs)


In [35]:
# Run RWA calculation function

[df_out, rwa_by_Insured_class, rwa_by_MRS_Bin, res_data]  = Lib_RWA.rwa_calculation(rwa_raw_data_TrueAddon, lgd_gen_floor, CMHC_lgd, CMHC_pd)



0
1
2


In [36]:
# Save Data to Parquet file

res_data.to_parquet(Output_dir +'\\'+'eqb_rwa_original.parquet', index=False)

# with pd.ExcelWriter(Output_dir +'\\'+'eqb_rwa_original.xlsx' )as writer:
#     res_data.to_excel(writer, sheet_name='RWA_Data', index=False)
#     df_out.to_excel(writer, sheet_name="RWA") 
#     rwa_by_Insured_class.to_excel(writer, sheet_name="rwa_by_Insured_class")
#     rwa_by_MRS_Bin.to_excel(writer, sheet_name="rwa_by_MRS_Bin")

# Sanity Check the Outputs

In [37]:
# print snapshot date 

print('Snapshot date: ' + snapshot_date) #20250403 George added

# print the shape of res_data

print('Shape of res_data: ' + str(res_data.shape)) #20250403 George added

# Group by Insured_Indicator, count number of Loan_Number, sum of EAD and RWA_new

print('***************Group by Insured_Indicator*****************') #20250403 George added

Sumry = res_data.groupby(['Insured_class'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'EAD': sum, 'RWA_new': sum}) #20250403 George added

# Add a total row to Sumry

Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added
Sumry.at[Sumry.index[-1], 'Insured_class'] = 'Total' #20250403 George added

# Add a column RWA_Ratio as the ratio of RWA_new and EAD

Sumry['RWA_Ratio'] = Sumry['RWA_new'] / Sumry['EAD'] #20250403 George added

# print the summary

print(Sumry) #20250403 George added

# print columns in alphabetical ored

print(sorted(res_data.columns)) #20250403 George added

Snapshot date: 2024-12-31
Shape of res_data: (55659, 57)
***************Group by Insured_Indicator*****************
  Insured_class  Loan_Number           EAD       RWA_new  RWA_Ratio
0            CG       4534.0  1.290269e+09  9.555135e+07   0.074055
1          CMHC      11040.0  3.061192e+09  5.133710e+07   0.016770
2         Sagen       8832.0  2.293171e+09  1.785676e+08   0.077869
3     Uninsured      28996.0  1.562863e+10  3.487959e+09   0.223177
4         Total      53402.0  2.227326e+10  3.813415e+09   0.171210
['AddOn', 'Advance_Amount', 'Alt_Prime_Indicator', 'Appraisal_Bin_WOE', 'BF_Appr_Prov_Dw', 'BF_LTV_Incl_Parter_Incl_HELOC_FSA_Dw', 'BF_LTV_Tot_Exp_FSA_Dw_WF', 'Base_Line_LGD', 'Beacon_Avg_App_CoApp', 'Beacon_Avg_App_CoApp_WOE', 'CalibratedPD', 'Combo_LTV_Insured_Ind', 'Combo_LTV_Insured_Ind_WOE', 'Combo_Province_Metro_Override', 'Combo_Province_Metro_Override_WOE', 'Corporate_Applicant_Ind', 'DLGD_floor', 'EAD', 'EAD_DLGD_Modified', 'EAD_Post_CRM', 'Final_LGD', 'Insured_I

C:\Users\georgez\AppData\Local\Temp\ipykernel_28156\1574102103.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Sumry = Sumry.append(Sumry.sum(numeric_only=True), ignore_index=True) #20250403 George added


In [38]:
# Group by Corporate_Applicant_Ind, count number of Loan_Number, sum of EAD and RWA_new

print('***************Group by Corporate_Applicant_Ind*****************') 

print(res_data.groupby(['Corporate_Applicant_Ind'], as_index=False).agg({'Loan_Number': pd.Series.nunique, 'EAD': sum, 'RWA_new': sum})) #20250403 George added

***************Group by Corporate_Applicant_Ind*****************
   Corporate_Applicant_Ind  Loan_Number           EAD       RWA_new
0                      0.0        54203  2.227180e+10  3.813310e+09
1                      1.0         1426  1.384430e+06  1.045012e+05


In [39]:
# For res_data, filter missing Insured_class loans

res_data_Insured_class_null = res_data.loc[res_data['Insured_class'].isnull()] 


